In [3]:
import scipy.io as io
import numpy as np

In [4]:
data = io.loadmat('PaviaU.mat')

In [5]:
dataset = data['paviaU']

In [6]:
labels = io.loadmat('PaviaU_gt.mat')

In [7]:
gt_labels = labels['paviaU_gt']

In [8]:
print(dataset[0][0])
print(gt_labels[0][0])

[ 647  499  464  371  291  319  365  322  296  305  277  219  219  222
  201  162  157  183  204  194  198  216  249  284  294  322  338  342
  336  342  362  365  348  341  324  316  293  274  251  244  263  248
  236  254  255  240  223  203  202  191  185  200  203  181  168  174
  185  187  165  151  144  140  149  169  192  227  287  376  493  633
  763  913 1149 1442 1759 2102 2425 2689 2895 3058 3196 3252 3195 3297
 3542 3550 3537 3545 3514 3477 3468 3433 3408 3420 3416 3335 3256 3226
 3205 3210 3221 3238 3250]
0


In [9]:
(x,y,z) = dataset.shape
print(x,y,z)

610 340 103


In [12]:
for i in range(x):
    for j in range(y):
        l = gt_labels[i][j]
        if l!=0:
            a[l].append(np.array(dataset[i][j]))

In [11]:
a = {x:[] for x in range(1,10)}

In [13]:
for i in range(1,10):
    print(len(a[i]))

6631
18649
2099
3064
1345
5029
1330
3682
947


In [14]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D , Flatten, MaxPooling2D

Using TensorFlow backend.


In [28]:
model = Sequential()
model.add(Conv2D(20, kernel_size=(11,1), activation='relu', input_shape=(103,1,1)))
model.add(MaxPooling2D(pool_size = (3,1)))
model.add(Flatten())
model.add(Dense(100,activation='relu'))
model.add(Dense(9,activation = 'softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 93, 1, 20)         240       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 31, 1, 20)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 620)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               62100     
_________________________________________________________________
dense_4 (Dense)              (None, 9)                 909       
Total params: 63,249
Trainable params: 63,249
Non-trainable params: 0
_________________________________________________________________


In [29]:
y_train = []
train = []
for i in range(1,10):
    for j in range(200):
        train.append(a[i][j])
        y_train.append(i)
train = np.array(train)
y_train = np.array(y_train)

In [30]:
y_test = []
test = []
for i in range(1,10):
    for j in range(200,len(a[i])):
        test.append(a[i][j])
        y_test.append(i)
test = np.array(test)
y_test = np.array(y_test)

In [31]:
test = test.reshape(len(test),103,1,1)

In [32]:
train = train.reshape(len(train),103,1,1)

In [33]:
from keras.utils import to_categorical

In [34]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [35]:
y_train = y_train[:,1:]
y_test = y_test[:,1:]

In [36]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics = ['accuracy'])

In [38]:
model.fit(test,y_test,validation_data = (train,y_train),epochs = 5)

Train on 40976 samples, validate on 1800 samples
Epoch 1/5
40976/40976 [==============================] - 5s 117us/step - loss: 8.8611 - acc: 0.4502 - val_loss: 14.3272 - val_acc: 0.1111
Epoch 2/5
40976/40976 [==============================] - 5s 116us/step - loss: 8.8611 - acc: 0.4502 - val_loss: 14.3272 - val_acc: 0.1111
Epoch 3/5
40976/40976 [==============================] - 5s 116us/step - loss: 8.8611 - acc: 0.4502 - val_loss: 14.3272 - val_acc: 0.1111
Epoch 4/5
40976/40976 [==============================] - 5s 119us/step - loss: 8.8611 - acc: 0.4502 - val_loss: 14.3272 - val_acc: 0.1111
Epoch 5/5
40976/40976 [==============================] - 5s 118us/step - loss: 8.8611 - acc: 0.4502 - val_loss: 14.3272 - val_acc: 0.1111


In [27]:
m = 0
for i in range(200,400):
    temp = max(train[i].reshape(103,1))
    if temp>m:
        m = temp
print(m)

[2547]
